<a href="https://www.kaggle.com/code/whsos927/notebook96856d2406?scriptVersionId=115969654" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [2]:
# load excel
data = pd.read_csv("/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [3]:
!pip install keras

     |████████████████████████████████| 1.7 MB 2.1 MB/s eta 0:00:01


In [4]:
# Loading Dependencies
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers

from tokenizers import BertWordPieceTokenizer

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from tensorflow.keras.preprocessing.sequence import pad_sequences


import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

2023-01-10 07:02:04.248638: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-01-10 07:02:04.248796: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.0.0.2:8470


2023-01-10 07:02:13.144353: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-01-10 07:02:13.147386: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-01-10 07:02:13.147430: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-10 07:02:13.147458: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0daa2c71af6f): /proc/driver/nvidia/version does not exist
2023-01-10 07:02:13.150814: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

REPLICAS:  8


In [6]:
# replace 1 and 0
data.sentiment = data.sentiment.replace({'positive': 1, 'negative': 0})

In [44]:
# Creating training and valid dataset by ratio of 1:9
# valid dataset
valid = data.sample(frac = 0.1)
 
# Creating dataframe with
# training dataset
train1 = data.drop(valid.index)
 
print("\n10% of the given DataFrame:")
print(valid.shape)
 
print("\nrest 90% of the given DataFrame:")
print(train1.shape)


10% of the given DataFrame:
(5000, 2)

rest 90% of the given DataFrame:
(45000, 2)


In [8]:
valid.sentiment.value_counts()

0    2522
1    2478
Name: sentiment, dtype: int64

In [9]:
train1.sentiment.value_counts()

1    22522
0    22478
Name: sentiment, dtype: int64

In [10]:
train1 = train1.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [11]:
valid.head()

,review,sentiment
0,"I love ghost stories in general, but I PARTICU...",1
1,Why do movie makers always go against the auth...,0
2,Even if this film was allegedly a joke in resp...,0
3,The Black Castle is one of those film's that h...,1
4,"Sandra Bernhard's Without You I'm Nothing, the...",1


In [12]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
    """
    Encoder for encoding the text into sequence of integers for BERT Input
    """
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [13]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192

In [14]:
# First load the real tokenizer
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# Save the loaded tokenizer locally
tokenizer.save_pretrained('.')
# Reload it with the huggingface tokenizers library
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [15]:
x_train = fast_encode(train1.review.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = fast_encode(valid.review.astype(str), fast_tokenizer, maxlen=MAX_LEN)
#x_test = fast_encode(test1.review.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = train1.sentiment.values
y_valid = valid.sentiment.values

100%|██████████| 20/20 [00:02<00:00,  9.54it/s]


In [16]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .shuffle(3000)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)

# test_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices(x_test)
#     .batch(BATCH_SIZE)
# )

**Metrics with Accuracy**

In [17]:
def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [20]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model_1 (TFDi TFBaseModelOutput(last_hi 134734080 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 769       
Total params: 134,734,849
Trainable params: 134,734,849
Non-trainable params: 0
_________________________________________________________________
CPU times: user 8.35 s, sys: 7.6 s, total: 16 s
Wall time: 16.4 s


In [21]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=6
    #batch_size = BATCH_SIZE
)

Epoch 1/6
351/351 [==============================] - 77s 102ms/step - loss: 0.5315 - accuracy: 0.7062 - val_loss: 0.3219 - val_accuracy: 0.8570
Epoch 2/6
351/351 [==============================] - 28s 79ms/step - loss: 0.3176 - accuracy: 0.8600 - val_loss: 0.2902 - val_accuracy: 0.8740
Epoch 3/6
351/351 [==============================] - 28s 79ms/step - loss: 0.2652 - accuracy: 0.8855 - val_loss: 0.2818 - val_accuracy: 0.8842
Epoch 4/6
351/351 [==============================] - 28s 79ms/step - loss: 0.2243 - accuracy: 0.9070 - val_loss: 0.2906 - val_accuracy: 0.8842
Epoch 5/6
351/351 [==============================] - 28s 80ms/step - loss: 0.1827 - accuracy: 0.9269 - val_loss: 0.2974 - val_accuracy: 0.8884
Epoch 6/6
351/351 [==============================] - 28s 79ms/step - loss: 0.1488 - accuracy: 0.9415 - val_loss: 0.3235 - val_accuracy: 0.8888


In [22]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Epoch 1/6
39/39 [==============================] - 3s 74ms/step - loss: 0.3013 - accuracy: 0.8784
Epoch 2/6
39/39 [==============================] - 29s 74ms/step - loss: 0.2286 - accuracy: 0.9087
Epoch 3/6
39/39 [==============================] - 3s 74ms/step - loss: 0.1788 - accuracy: 0.9298
Epoch 4/6
39/39 [==============================] - 3s 74ms/step - loss: 0.1407 - accuracy: 0.9456
Epoch 5/6
39/39 [==============================] - 3s 74ms/step - loss: 0.1049 - accuracy: 0.9598
Epoch 6/6
39/39 [==============================] - 3s 74ms/step - loss: 0.0851 - accuracy: 0.9690


**Metrics with recall**

In [28]:
from tensorflow.keras import backend as K
#tensorflow.keras.
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [29]:
def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=[recall_m])
    
    return model

In [30]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model_3 (TFDi TFBaseModelOutput(last_hi 134734080 
_________________________________________________________________
tf.__operators__.getitem_2 ( (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 769       
Total params: 134,734,849
Trainable params: 134,734,849
Non-trainable params: 0
_________________________________________________________________
CPU times: user 6.47 s, sys: 5.74 s, total: 12.2 s
Wall time: 13.4 s


In [31]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=6
    #batch_size = BATCH_SIZE
)

Epoch 1/6
351/351 [==============================] - 81s 103ms/step - loss: 0.5414 - recall_m: 0.6677 - val_loss: 0.3220 - val_recall_m: 0.8007
Epoch 2/6
351/351 [==============================] - 28s 80ms/step - loss: 0.3157 - recall_m: 0.8628 - val_loss: 0.2890 - val_recall_m: 0.8551
Epoch 3/6
351/351 [==============================] - 28s 80ms/step - loss: 0.2653 - recall_m: 0.8915 - val_loss: 0.3085 - val_recall_m: 0.9223
Epoch 4/6
351/351 [==============================] - 29s 82ms/step - loss: 0.2263 - recall_m: 0.9148 - val_loss: 0.2941 - val_recall_m: 0.8818
Epoch 5/6
351/351 [==============================] - 28s 80ms/step - loss: 0.1871 - recall_m: 0.9305 - val_loss: 0.3055 - val_recall_m: 0.8965
Epoch 6/6
351/351 [==============================] - 28s 79ms/step - loss: 0.1527 - recall_m: 0.9414 - val_loss: 0.3319 - val_recall_m: 0.8909


In [32]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Epoch 1/6
39/39 [==============================] - 3s 75ms/step - loss: 0.3093 - recall_m: 0.8816
Epoch 2/6
39/39 [==============================] - 29s 75ms/step - loss: 0.2369 - recall_m: 0.8893
Epoch 3/6
39/39 [==============================] - 3s 75ms/step - loss: 0.1904 - recall_m: 0.9131
Epoch 4/6
39/39 [==============================] - 3s 74ms/step - loss: 0.1609 - recall_m: 0.9242
Epoch 5/6
39/39 [==============================] - 3s 74ms/step - loss: 0.1386 - recall_m: 0.9363
Epoch 6/6
39/39 [==============================] - 3s 74ms/step - loss: 0.1068 - recall_m: 0.9454


**Metrics with F1 Score**

In [39]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [40]:
def build_model(transformer, max_len=512):
    """
    function for training the BERT model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=[f1_m])
    
    return model

In [41]:
%%time
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel
        .from_pretrained('distilbert-base-multilingual-cased')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model_5 (TFDi TFBaseModelOutput(last_hi 134734080 
_________________________________________________________________
tf.__operators__.getitem_4 ( (None, 768)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 769       
Total params: 134,734,849
Trainable params: 134,734,849
Non-trainable params: 0
_________________________________________________________________
CPU times: user 6.26 s, sys: 4.69 s, total: 11 s
Wall time: 11 s


In [42]:
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=6
    #batch_size = BATCH_SIZE
)

Epoch 1/6
351/351 [==============================] - 82s 103ms/step - loss: 0.5646 - f1_m: 0.6430 - val_loss: 0.3300 - val_f1_m: 0.8406
Epoch 2/6
351/351 [==============================] - 28s 80ms/step - loss: 0.3205 - f1_m: 0.8503 - val_loss: 0.3043 - val_f1_m: 0.8383
Epoch 3/6
351/351 [==============================] - 29s 82ms/step - loss: 0.2694 - f1_m: 0.8761 - val_loss: 0.2808 - val_f1_m: 0.8607
Epoch 4/6
351/351 [==============================] - 28s 81ms/step - loss: 0.2269 - f1_m: 0.9016 - val_loss: 0.2876 - val_f1_m: 0.8708
Epoch 5/6
351/351 [==============================] - 28s 80ms/step - loss: 0.1869 - f1_m: 0.9184 - val_loss: 0.3099 - val_f1_m: 0.8673
Epoch 6/6
351/351 [==============================] - 28s 80ms/step - loss: 0.1521 - f1_m: 0.9353 - val_loss: 0.3425 - val_f1_m: 0.8681


In [43]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Epoch 1/6
39/39 [==============================] - 3s 75ms/step - loss: 0.3021 - f1_m: 0.8732
Epoch 2/6
39/39 [==============================] - 27s 75ms/step - loss: 0.2220 - f1_m: 0.8938
Epoch 3/6
39/39 [==============================] - 3s 75ms/step - loss: 0.1814 - f1_m: 0.9131
Epoch 4/6
39/39 [==============================] - 3s 74ms/step - loss: 0.1353 - f1_m: 0.9337
Epoch 5/6
39/39 [==============================] - 3s 74ms/step - loss: 0.1060 - f1_m: 0.9433
Epoch 6/6
39/39 [==============================] - 3s 74ms/step - loss: 0.0766 - f1_m: 0.9541
